In [ ]:
# Gradio demo
import gradio as gr
import pandas as pd
import textdistance

# Updated fuzzy function
def thefuzz_distance(needle, haystack_df, algorithm='jaccard'):
    # Get the algorithm function from textdistance
    algo_func = getattr(textdistance, algorithm)
    # Flatten the DataFrame to a list
    haystack = haystack_df.values.flatten().tolist()
    
    # Check if needle is a DataFrame
    if isinstance(needle, pd.DataFrame):
        needle_list = needle.values.flatten().tolist()
    else:
        needle_list = [needle]
    
    results = []
    for needle_item in needle_list:
        # Calculate similarity scores with normalization
        scores = [(index, algo_func.normalized_similarity(needle_item, item)) for index, item in enumerate(haystack)]
        # Sort based on scores in descending order
        scores.sort(key=lambda x: x[1], reverse=True)
        # Append the top index and score to results as a list
        results.append(list(scores[0]))
    
    return results

# List of algorithms to test
algorithms = [
    'jaccard', 'levenshtein', 'hamming', 'cosine', 'jaro', 'jaro_winkler', 
    'sorensen', 'ratcliff_obershelp', 'damerau_levenshtein', 'strcmp95', 
    'needleman_wunsch', 'smith_waterman', 'tversky', 'overlap', 'monge_elkan',
    'lcsseq', 'lcsstr', 'gotoh', 'sorensen_dice', 'dice', 'bag', 'editex', 
    'mlipns', 'mra'
]

# Create the Gradio interface
demo = gr.Interface(
    fn=thefuzz_distance,
    inputs=[
        gr.Textbox(label="lookup_value"),
        gr.Dataframe(headers=["items"], label="lookup_array", type="pandas"),  # Set type to pandas DataFrame
        gr.Dropdown(choices=algorithms, label="algorithm")
    ],
    outputs=gr.Dataframe(headers=["Index", "Score"], label="Result"),  # Change output to Dataframe
    examples = [
        ["apple", pd.DataFrame({"items": ["apples", "apply", "ape", "grape"]}), "jaccard"],
        ["cat", pd.DataFrame({"items": ["cats", "cut", "bat", "hat"]}), "levenshtein"],
        ["hello", pd.DataFrame({"items": ["helloo", "helo", "hallo", "hey"]}), "cosine"]
    ]
)

# Launch the interface
demo.launch(show_error=True)